In [2]:
from pyscipopt import Model
import numpy as np

# Practica 1

## Ejercicio 1

Indices
- Cantidad de porciones: con i perteneciente a [1, 2, 3]
- Cantidad de vitaminas: con j perteneciente a [A, C, D]

Variables
- A_i - Cantidad de porciones de 30g del alimento i 

Datos
- V_ij - Cantidad de vitamina j en una porcion de 30 gramos del alimento i
- C_i - Costo de 30 gramos del alimento i
- RDM_j - racion diaria minima de vitamina j

FO
- MIN(COSTO = SUM(A_i * C_I)) - minimizar costo de la porcion

s.a
- SUM_i(A_i) >= 225/30  - la porcion tiene que tener al menos 225 grm
- SUM_i(A_I * V_ij) >= RDM_j for all j - la porcion tiene que tener todas las vitaminas minimas

In [13]:
# Vitamin content per 30g [rows: A, C, D; columns: Alimentos 1, 2, 3]
vitamin_content = np.array([
    [0.03, 0.02, 0.04],   # Vitamina A
    [0.01, 0.015, 0.005], # Vitamina C
    [0.04, 0.03, 0.02]    # Vitamina D
])

# Ración Diaria Mínima (RDM) for vitamins A, C, D
rdm = np.array([0.3, 0.12, 0.21])

# Cost per 30g portion for each food
cost = np.array([0.15, 0.10, 0.12])

# Portion size per unit and total minimum required in grams
portion_size = 30
min_total_grams = 225
min_total_portions = min_total_grams / portion_size  # 7.5 portions minimum




model = Model('Ej1')

food_portion = np.empty(len(cost), dtype='object')
for i in range(len(cost)):
  food_portion[i] = model.addVar(f'x_{i}', vtype='I')

model.setObjective(sum(food_portion[i] * cost[i] for i in range(len(cost))), sense='minimize')

model.addCons(sum(food_portion[i] for i in range(len(cost))) >= min_total_portions)

for j in range(len(rdm)):
    model.addCons(sum(food_portion[i] * vitamin_content[j, i] for i in range(len(cost))) >= rdm[j])

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej1.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 3.700000e+04
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 3 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 6 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 6 tightened bounds, 0 added holes, 1 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3 variables (0 bin, 3 int, 0 impl, 0 cont) and 4 constraints
      4 constraints of type <linear>
transformed objective value is always integral (scale: 0.01)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |va

In [23]:
print(f"costo final: {sum(sol[food_portion[i]] * cost[i] for i in range(len(food_portion)))}")
print(f"req: {225/30}; cantidad porciones: {sum(sol[food_portion[i]] for i in range(len(food_portion)))}")

for j in range(len(rdm)):
    print(f"req: {rdm[j]}; vitamina {j}: {sum(sol[food_portion[i]] * vitamin_content[j,i] for i in range(len(food_portion)))}")

costo final: 1.1800000000000002
req: 7.5; cantidad porciones: 11.0
req: 0.3; vitamina 0: 0.30000000000000004
req: 0.12; vitamina 1: 0.125
req: 0.21; vitamina 2: 0.29


## Ejercicio 2

Indices
- Cantidad de proyectos: con i perteneciente a [1, .., 6]

Variables
- A_i - Cantidad de millones de pesos invertidos en el proyecto i

Datos
- c_i - categoria de proyecto i
- U_i - utilidad del proyecto i
- fm_i - financiamiento maximo del proyecto i

FO
- MAX(utilidad = SUM(A_i * U_i)) - maximizar utilidad

s.a
- SUM_i(A_i) <= 1000 - el presupuesto total debe ser menor al disponible
- A_i <= fm_i - el financiamiento de cada proyecto no puede exceder el maximo
- A_i >= 50/100 * fm_i for all i in nuclear - el financiamiento de los proyectos nucleares tiene que ser al menos del 50%
- SUM_i(A_i) >= 300 for all i in solar - el financiamiento de proyectos nucleares tiene que ser al menos de 300 M

In [31]:
# Net utility per peso invested for each project
utility = np.array([4.4, 3.8, 4.1, 3.5, 5.1, 3.2])

# Required total financing for each project (in millions)
max_financing = np.array([220, 180, 250, 150, 400, 120])

# Total available budget (in millions)
budget = 1000

# Indices for project categories
solar_index = [0, 1]     # Projects 1 and 2
nuclear_index = [4]          # Project 5



model = Model('Ej2')

project_range = range(len(utility))
project_cash = np.empty(len(utility), dtype='object')
for i in project_range:
  project_cash[i] = model.addVar(f'x_{i}', vtype='I')

model.setObjective(sum(project_cash[i] * utility[i] for i in project_range), sense='maximize')

model.addCons(sum(project_cash[i] for i in project_range) <= 1000)

for i in project_range:
    model.addCons(project_cash[i] <= max_financing[i])
    if i in nuclear_index:
        model.addCons(project_cash[i] >= 50/100 * max_financing[i])

model.addCons(sum(project_cash[i] for i in project_range if i in solar_index) >= 300)

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej2.sol')

presolving:
(round 1, fast)       0 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 0 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 1 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 4, fast)       1 del vars, 8 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (5 rounds: 5 fast, 3 medium, 3 exhaustive):
 1 deleted vars, 8 deleted constraints, 0 added constraints, 15 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 5 variables (0 bin, 5 int, 0 impl, 0 cont) and 1 constraints
   

In [32]:
print(f"utilidad final: {sum(sol[project_cash[i]] * utility[i] for i in project_range)}")
print(f"req: {1000}; budget gastado: {sum(sol[project_cash[i]] for i in project_range)}")
print(f"req: {0.5 * sum(max_financing[i] for i in nuclear_index)}; financing nuclear: {sum(sol[project_cash[i]] for i in nuclear_index)}")
print(f"req: {300}; financing solar: {sum(sol[project_cash[i]] for i in solar_index)}")

utilidad final: 4527.0
req: 1000; budget gastado: 1000.0
req: 200.0; financing nuclear: 400.0
req: 300; financing solar: 350.0


## Ejercicio 3

Indices
- Cantidad de paises: con i perteneciente a [1, .., N]
- Cantidad de colores: con k perteneciente a [1, .., inf a priori, por teo de 4 colores sabemos q son 4]

Variables
- C_k: uso el color i
- P_ik: pinto el pais i del color k

Datos
- L_ij - flag que dice si el pais i es limitrofe al pais j (matriz simetrica)

FO
- MIN(cantidad_de_colores_usados = SUM_K(C_k)) - minimizar la cantidad de colores usados

s.a
- eps * SUM_i(P_ik) <= C_k for all k - si pinto algun pais del color k, C_k tiene que valer 1, aca el eps puede vale N
- SUM_k(P_ik) = 1 for all i - tengo que pintar todos los paises de un solo color
- P_ik + P_jk <= 1 for all (i,j) tal que son limitrofes for all k - no puedo pintar limitrofes del mismo color